# Multi Model Building (Batch 3)

In [1]:
FITTING_BATCH_NUM = 3

In [2]:
# Remove TF logging warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
import pandas as pd
import tensorflow as tf
import keras
from keras import layers
from model_helper_functions import create_fit_and_save_model, send_ifttt_notification, update_top_values
from data_pipeline import prep_data, get_train_val_test_data
from itertools import product
import re
import pathlib
import shutil

# Import correct optimizer
import platform
if platform.system() == "Darwin" and platform.processor() == "arm":
    from keras.optimizers.legacy import RMSprop, Adam
else:
    from keras.optimizers import RMSprop, Adam

In [4]:
# Remove TF logging warnings
tf.get_logger().setLevel('ERROR')

In [5]:
tf.random.set_seed(15)
keras.utils.set_random_seed(15)

In [6]:
data_dir = '../bears'
df = prep_data(data_dir)
train_df, val_df, test_df = get_train_val_test_data(df)

Training set: 70%, Validation set: 22.5%, Test set: 7.5%


In [7]:
image_count = len(list(pathlib.Path(data_dir).glob('*/*')))
print('Total image count:',image_count)
print('Image count equal to dataframe length?', image_count == len(df))

Total image count: 288
Image count equal to dataframe length? True


In [8]:
num_train_samples = len(train_df)
num_val_samples = len(val_df)
num_test_samples = len(test_df)
print('Number of training samples:',num_train_samples)
print('Number of validation samples:',num_val_samples)
print('Number of test samples:',num_test_samples)

Number of training samples: 201
Number of validation samples: 65
Number of test samples: 22


---

## Hyperparameter Grid

In [9]:
g_batch_size = [4]
g_epochs = [35]
g_augmentation_params = [
    # either None or (flip, rotate_factor, zoom_factor, random_flip_str)
    None,
]
g_cnn_params = [
    # cnn_units, cnn_filters, cnn_strides
    [(32,), [(3,3)], [(1,1)]],
    [(32, 64), [(3,3), (3,3)], [(1,1), (1,1)]],
    [(32, 64, 128), [(3,3), (3,3), (3,3)], [(1,1), (1,1), (1,1)]],
]
g_dropout = [0]
g_dense_units = [
    (64, 3),
    (128, 3)
]
g_activation = ['relu', 'sigmoid']
g_optimizer = [RMSprop]
g_earlystop_patience = [10]
g_reducel_patience = [2]

In [10]:
all_combinations = list(product(g_batch_size, g_epochs, g_augmentation_params, g_cnn_params, g_dropout,
                                g_dense_units, g_activation, g_optimizer, g_earlystop_patience, g_reducel_patience))
print(f'There are {len(all_combinations)} parameter combinations to run.')

There are 12 parameter combinations to run.


In [11]:
# Keep track of model fitting in order to resume at a later time if needed.
progress_file = f'./model_checkpoints_{FITTING_BATCH_NUM}/model_building_progress.csv'
os.makedirs(f'./model_checkpoints_{FITTING_BATCH_NUM}/', exist_ok=True)
if os.path.exists(progress_file):
    print('Using existing progress file.')
    progress = pd.read_csv(progress_file)
else:
    print('Creating new progress file.')
    progress = pd.DataFrame.from_records(all_combinations, columns=['batch_size', 'epochs', 'augmentation_params', 'cnn_params',
                                                                    'dropout', 'dense_units', 'activation', 'optimizer',
                                                                    'earlystop_patience', 'reducel_patience'])
    progress.insert(0, 'model', range(len(all_combinations)))
    progress['finished'] = False
    class_names = '(Adam|RMSprop)'
    progress['optimizer'] = progress['optimizer'].apply(lambda x: re.sub(f'.*{class_names}.*','\\1',str(x)))
    
    progress.to_csv(progress_file, index=False)
    progress = pd.read_csv(progress_file)

Creating new progress file.


In [12]:
display(progress)

,model,batch_size,epochs,augmentation_params,cnn_params,dropout,dense_units,activation,optimizer,earlystop_patience,reducel_patience,finished
0,0,4,35,NaN,"[(32,), [(3, 3)], [(1, 1)]]",0,"(64, 3)",relu,RMSprop,10,2,False
1,1,4,35,NaN,"[(32,), [(3, 3)], [(1, 1)]]",0,"(64, 3)",sigmoid,RMSprop,10,2,False
2,2,4,35,NaN,"[(32,), [(3, 3)], [(1, 1)]]",0,"(128, 3)",relu,RMSprop,10,2,False
3,3,4,35,NaN,"[(32,), [(3, 3)], [(1, 1)]]",0,"(128, 3)",sigmoid,RMSprop,10,2,False
4,4,4,35,NaN,"[(32, 64), [(3, 3), (3, 3)], [(1, 1), (1, 1)]]",0,"(64, 3)",relu,RMSprop,10,2,False
5,5,4,35,NaN,"[(32, 64), [(3, 3), (3, 3)], [(1, 1), (1, 1)]]",0,"(64, 3)",sigmoid,RMSprop,10,2,False
6,6,4,35,NaN,"[(32, 64), [(3, 3), (3, 3)], [(1, 1), (1, 1)]]",0,"(128, 3)",relu,RMSprop,10,2,False
7,7,4,35,NaN,"[(32, 64), [(3, 3), (3, 3)], [(1, 1), (1, 1)]]",0,"(128, 3)",sigmoid,RMSprop,10,2,False
8,8,4,35,NaN,"[(32, 64, 128), [(3, 3), (3, 3), (3, 3)], [(1,...",0,"(64, 3)",relu,RMSprop,10,2,False
9,9,4,35,NaN,"[(32, 64, 128), [(3, 3), (3, 3), (3, 3)], [(1,...",0,"(64, 3)",sigmoid,RMSprop,10,2,False


---

## Fit Models

In [13]:
os.makedirs(f'./model_checkpoints_{FITTING_BATCH_NUM}/val_metrics', exist_ok=True)
os.makedirs(f'./model_checkpoints_{FITTING_BATCH_NUM}/model_histories', exist_ok=True)
top_val_acc = dict()
models = []
for i, params in enumerate(all_combinations):
    if progress.at[i, 'finished']:
        print(f'Model {i} has already been fitted.')
    else:
        try:
            val_metrics = create_fit_and_save_model(f'model{i}', train_df, val_df, test_df, params, FITTING_BATCH_NUM)
            progress.at[i, 'finished'] = True
            progress.to_csv(progress_file, index=False)
            print(f"""Model {i}: validation accuracy= {val_metrics['accuracy']:.4f}, ran epochs= {val_metrics['ran_epochs']}, best epoch= {val_metrics['best_epoch']}, time={val_metrics['total_time']/60:.2f}min.""")
            
            update_top_values(top_val_acc, val_metrics['accuracy'], i, FITTING_BATCH_NUM)
                
        except:
            print(f'ERROR fitting model {i}')

Model 0: validation accuracy= 0.7846, ran epochs= 25, best epoch= 15, time=2.45min.


Model 1: validation accuracy= 0.3385, ran epochs= 11, best epoch= 1, time=1.06min.


Model 2: validation accuracy= 0.8462, ran epochs= 29, best epoch= 19, time=3.93min.


Model 3: validation accuracy= 0.3385, ran epochs= 11, best epoch= 1, time=1.45min.


Model 4: validation accuracy= 0.7385, ran epochs= 21, best epoch= 11, time=1.75min.


...

Model 7: validation accuracy= 0.8769, ran epochs= 34, best epoch= 24, time=3.51min.


Model 8: validation accuracy= 0.7846, ran epochs= 21, best epoch= 11, time=1.74min.


Model 9: validation accuracy= 0.8923, ran epochs= 25, best epoch= 15, time=2.06min.


Model 10: validation accuracy= 0.7846, ran epochs= 24, best epoch= 14, time=2.17min.


Model 11: validation accuracy= 0.8769, ran epochs= 31, best epoch= 21, time=2.79min.


In [14]:
send_ifttt_notification('finished')

Notification sent


---